In [ ]:
import json
import re
import string

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.neighbors import NearestNeighbors

## Data preprocessing

In [ ]:
books_df = pd.read_csv('books_enriched.csv')
user_ratings_df = pd.read_csv('ratings.csv')

In [ ]:
print(f"Number of books: {books_df.shape[0]}")
print(books_df.info())
display(books_df.describe())
print(books_df.isnull().sum())

In [ ]:
print(user_ratings_df.info())
print(f"\nNumber of missing values:\n{user_ratings_df.isnull().sum()}")

In [ ]:
def format_author_names(authors_str):
    try:
        authors_list = json.loads(authors_str.replace("'", '"'))
        return ', '.join(authors_list)
    except json.JSONDecodeError:
        return authors_str

In [ ]:
books_df.drop(columns=['Unnamed: 0', 'index'], inplace=True)

books_df['description'] = books_df['description'].fillna('')
books_df['isbn'] = books_df['isbn'].fillna('Unknown')
books_df['isbn13'] = books_df['isbn13'].fillna(0).astype(int)
books_df['original_publication_year'] = books_df['original_publication_year'].fillna(0).astype(int)
books_df['original_title'] = books_df['original_title'].fillna('')
books_df['pages'] = books_df['pages'].fillna(0).astype(int)
books_df['authors'] = books_df['authors'].apply(format_author_names)
books_df['title'] = books_df['title'].str.replace(r"\s+", " ", regex=True)

In [ ]:
def normalize_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

books_df['normalized_title'] = books_df['title'].apply(normalize_text)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix_titles = tfidf_vectorizer.fit_transform(books_df['normalized_title'])

In [ ]:
def search_similar_books_by_title(query, df=books_df, tfidf_matrix=tfidf_matrix_titles, top_n=12, similarity_threshold=0.1):
    processed = re.sub("[^a-zA-Z0-9 ]", "", query.lower())
    query_vec = tfidf_vectorizer.transform([processed])

    cosine_similarities = linear_kernel(query_vec, tfidf_matrix).flatten()
    similar_indices = cosine_similarities.argsort()[::-1]
    filtered_indices = [idx for idx in similar_indices if cosine_similarities[idx] >= similarity_threshold]
    top_indices = filtered_indices[:top_n]
    if not top_indices:
        return pd.DataFrame()

    result_df = df.iloc[top_indices]
    result_df = result_df[['book_id', 'title', 'authors', 'average_rating', 'ratings_count', 'image_url', 'pages']]
    return result_df

In [ ]:
similar_books = search_similar_books_by_title("1984")
display(similar_books)

## Collaborative filtering recommendation system using a k-NN model

In [ ]:
# Creating a user-item matrix
user_item_matrix = user_ratings_df.pivot(index='user_id', columns='book_id', values='rating').fillna(0)

# Transform the matrix to a scipy sparse matrix
user_item_matrix_sparse = csr_matrix(user_item_matrix.values)

# Transpose the matrix so that books are rows
user_item_matrix_sparse_T = user_item_matrix_sparse.transpose()

# Train model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
model_knn.fit(user_item_matrix_sparse_T)

In [ ]:
book_id_to_idx = {book_id: idx for idx, book_id in enumerate(user_item_matrix.columns)}

def collaborative_recommendations(book_id, top_n=10):
    if book_id not in book_id_to_idx:
        return pd.DataFrame()
    book_idx = book_id_to_idx[book_id]

    distances, indices = model_knn.kneighbors(user_item_matrix_sparse_T[book_idx].reshape(1, -1), n_neighbors=top_n+1)

    book_indices = [user_item_matrix.columns[i] for i in indices.flatten()[1:]]

    return books_df[books_df['book_id'].isin(book_indices)]

## Content-based recommendation system using TF-IDF

In [ ]:
books_df['content'] = (pd.Series(books_df[['authors', 'title', 'genres', 'description']]
                                  .fillna('')
                                  .values.tolist()
                                  ).str.join(' '))

tf_content = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0., stop_words='english')
tfidf_matrix = tf_content.fit_transform(books_df['content'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
def content_based_recommendations(book_id, top_n=10, cosine_sim=cosine_sim, df=books_df):
    if book_id not in df['book_id'].values:
        return pd.DataFrame()

    book_id_to_index = pd.Series(df.index, index=df['book_id'])
    
    idx = book_id_to_index[book_id]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    top_n = min(top_n, len(sim_scores) - 1)
    sim_scores = sim_scores[1:top_n+1]

    book_indices = [i[0] for i in sim_scores]

    return df.iloc[book_indices]

In [29]:
def improved_recommendation(book_id, top_n=10, cosine_sim=cosine_sim, df=books_df):
    if book_id not in df['book_id'].values:
        return pd.DataFrame()

    book_id_to_index = pd.Series(df.index, index=df['book_id'])
    
    idx = book_id_to_index[book_id]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    top_n = min(top_n, len(sim_scores) - 1)
    sim_scores = sim_scores[1:top_n+1]
    
    book_indices = [i[0] for i in sim_scores]
    books_subset = df.iloc[book_indices][['book_id', 'title', 'authors', 'average_rating', 'ratings_count']]

    v = books_subset['ratings_count']
    m = books_subset['ratings_count'].quantile(0.75)
    R = books_subset['average_rating']
    C = books_subset['average_rating'].median()
    books_subset['new_score'] = (v / (v + m) * R) + (m / (m + v) * C)

    high_rating = books_subset[books_subset['ratings_count'] >= m]
    high_rating = high_rating.sort_values('new_score', ascending=False)

    return df.loc[high_rating.index]


In [30]:
improved_recommendation(13)

,authors,average_rating,best_book_id,book_id,books_count,description,genres,goodreads_book_id,image_url,isbn,...,ratings_5,ratings_count,small_image_url,title,work_id,work_ratings_count,work_text_reviews_count,authors_2,normalized_title,content
795,"George Orwell, Christopher Hitchens",4.26,5472,846,51,This edition features George Orwell's best kno...,"['classics', 'fiction', 'science-fiction', 'fa...",5472,https://images.gr-assets.com/books/1327959366m...,151010269,...,57179,116197,https://images.gr-assets.com/books/1327959366s...,Animal Farm / 1984,2966408,118761,1293,"['George Orwell', 'Christopher Hitchens']",animal farm 1984,"George Orwell, Christopher Hitchens Animal Far..."
604,"Haruki Murakami, Jay Rubin, Philip Gabriel",3.89,10357575,649,107,The year is 1984 and the city is Tokyo.A young...,"['fiction', 'fantasy', 'science-fiction', 'con...",10357575,https://images.gr-assets.com/books/1483103331m...,307593312,...,44559,125195,https://images.gr-assets.com/books/1483103331s...,1Q84,18160093,138691,13779,"['Haruki Murakami', 'Jay Rubin', 'Philip Gabri...",1q84,"Haruki Murakami, Jay Rubin, Philip Gabriel 1Q8..."
8168,"[George Orwell, Boris Grabnar, Celâl Üster, Ce...",3.87,7613,14,896,"“All animals are equal, but some animals are m...","['classics', 'fiction', 'fantasy', 'science-fi...",7613,https://images.gr-assets.com/books/1424037542m...,452284244,...,648912,1881700,https://images.gr-assets.com/books/1424037542s...,Animal Farm,2207778,1982987,35472,"['[George Orwell', 'Boris Grabnar', 'Celâl Üst...",animal farm,"[George Orwell, Boris Grabnar, Celâl Üster, Ce..."


## Hybrid model, combining both systems

In [ ]:
def hybrid_recommendations(book_id, top_n=10):
    content_recommendations = improved_recommendation(book_id, top_n=top_n)
    collab_recommendations = collaborative_recommendations(book_id, top_n=top_n)
    hybrid_recommendations_df = pd.concat([content_recommendations, collab_recommendations]).drop_duplicates().head(top_n)
    return hybrid_recommendations_df

In [ ]:
recommended_books = hybrid_recommendations(book_id=13, top_n=20)
recommended_books.shape[0]
recommended_books

## Recommender query testing

In [ ]:
# recommended_books = hybrid_recommendations(book_id=6, top_n=5)
recommended_books = content_based_recommendations(book_id=13, top_n=5)
json.loads(recommended_books[['book_id', 'title', 'authors', 'average_rating', 'ratings_count', 'image_url', 'description', 'genres']].to_json(orient='records'))

In [ ]:
def get_recommendations_as_json(book_id, model_type='hybrid', top_n=20):
    columns_to_include = ['book_id', 'title', 'authors', 'average_rating', 'ratings_count', 'image_url', 'pages']
    if model_type == 'hybrid':
        recommendations_df = hybrid_recommendations(book_id, top_n)
    elif model_type == 'collaborative':
        recommendations_df = collaborative_recommendations(book_id, top_n)
    elif model_type == 'content':
        recommendations_df = content_based_recommendations(book_id, top_n)
    elif model_type == 'improved':
        recommendations_df = improved_recommendation(book_id, top_n)
    else:
        return "Invalid model type"

    recommendations_df = recommendations_df[columns_to_include]
    # Convert DataFrame to JSON
    recommendations_json = recommendations_df.to_json(orient='records')
    return recommendations_json

# Example usage
json_output = get_recommendations_as_json(book_id=13, model_type='improved', top_n=10)
json.loads(json_output)